In [8]:
# To upload the kaggle.json file
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"harshikutty","key":"18db70790316a26eb1a34aa1e79a33a1"}'}

In [9]:
!mkdir ~/.kaggle

# Changing the prath to the .kaggle folder
!cp kaggle.json ~/.kaggle/
#  Changing the permissions to perform read and write access
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
# Downloading the dogs-vs-cats dataset
!kaggle competitions download -c dogs-vs-cats


 98% 795M/812M [00:03<00:00, 299MB/s]
100% 812M/812M [00:03<00:00, 218MB/s]


In [11]:
# Unzipping dogs-vs-cats dataset file
!unzip -qq dogs-vs-cats.zip

In [12]:
# Unzipping train sample
!unzip -qq train.zip

In [13]:
!pip install tensorflow==2.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1

In [17]:
# Creating directories and assiging images to training, validation and test directories
import os, shutil, pathlib

shutil.rmtree("cats_vs_dogs_small")

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [18]:
# to build and to run model summary:

from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                         

In [19]:
# to Configure the model
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [20]:
# to declare the image size and batch size to read the images from train. validation and test directories
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [21]:
# to use the callbacks function to monitor validation loss and running the model
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 18s 92ms/step - loss: 0.7159 - accuracy: 0.5075 - val_loss: 0.6929 - val_accuracy: 0.5600
Epoch 2/30
63/63 [==============================] - 6s 84ms/step - loss: 0.6933 - accuracy: 0.5260 - val_loss: 0.6921 - val_accuracy: 0.5020
Epoch 3/30
63/63 [==============================] - 4s 63ms/step - loss: 0.6970 - accuracy: 0.5470 - val_loss: 0.6839 - val_accuracy: 0.5530
Epoch 4/30
63/63 [==============================] - 4s 67ms/step - loss: 0.6778 - accuracy: 0.5755 - val_loss: 0.7516 - val_accuracy: 0.5360
Epoch 5/30
63/63 [==============================] - 7s 97ms/step - loss: 0.6518 - accuracy: 0.6210 - val_loss: 0.6542 - val_accuracy: 0.6210
Epoch 6/30
63/63 [==============================] - 4s 60ms/step - loss: 0.6032 - accuracy: 0.6875 - val_loss: 0.5859 - val_accuracy: 0.7000
Epoch 7/30
63/63 [==============================] - 4s 59ms/step - loss: 0.5699 - accuracy: 0.7140 - val_loss: 0.5992 - val_accuracy: 0.6800
Epoch 8/30
6

In [22]:
# To test the model
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 3s 34ms/step - loss: 0.5733 - accuracy: 0.6935
Test accuracy: 0.693


In [23]:
# To declare the Data Augumentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [24]:
# Building the model and configuing it
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [25]:
# Using the callbacks function to monitor validation loss and running the model
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/100
63/63 [==============================] - 6s 63ms/step - loss: 0.7112 - accuracy: 0.5145 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 2/100
63/63 [==============================] - 6s 97ms/step - loss: 0.6931 - accuracy: 0.5175 - val_loss: 1.0173 - val_accuracy: 0.5000
Epoch 3/100
63/63 [==============================] - 4s 58ms/step - loss: 0.6965 - accuracy: 0.5130 - val_loss: 0.6867 - val_accuracy: 0.5660
Epoch 4/100
63/63 [==============================] - 5s 82ms/step - loss: 0.6809 - accuracy: 0.5475 - val_loss: 0.6825 - val_accuracy: 0.5120
Epoch 5/100
63/63 [==============================] - 5s 78ms/step - loss: 0.6474 - accuracy: 0.6170 - val_loss: 0.6643 - val_accuracy: 0.5740
Epoch 6/100
63/63 [==============================] - 4s 59ms/step - loss: 0.6326 - accuracy: 0.6410 - val_loss: 0.6049 - val_accuracy: 0.6700
Epoch 7/100
63/63 [==============================] - 4s 62ms/step - loss: 0.6171 - accuracy: 0.6590 - val_loss: 0.6044 - val_accuracy: 0.6820
Epoch 

In [26]:
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 3s 36ms/step - loss: 0.4576 - accuracy: 0.8145
Test accuracy: 0.814


In [ ]:
Question 2:

In [28]:
# Creating directories and assiging images to training, validation and test directories
# Increasing the training sample from 1000 to 2000
import os
import shutil
import pathlib

#shutil.rmtree("cats_vs_dogs_small_IncreasedTrainSample")
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_IncreasedTrainSample")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

# Training has 2000 samples, test has 1000 samples, and validation has 500 samples
make_subset("train", start_index=0, end_index=2500)
make_subset("validation", start_index=2500, end_index=3000)
make_subset("test", start_index=3000, end_index=4000)

In [29]:
# Building the model
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [30]:
# to configure the model
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [31]:
# to use the callbacks function to monitor validation loss and running the model
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 7s 82ms/step - loss: 0.6967 - accuracy: 0.5080 - val_loss: 0.6909 - val_accuracy: 0.5000
Epoch 2/30
63/63 [==============================] - 5s 80ms/step - loss: 0.6976 - accuracy: 0.5660 - val_loss: 0.6724 - val_accuracy: 0.6090
Epoch 3/30
63/63 [==============================] - 4s 59ms/step - loss: 0.6598 - accuracy: 0.6075 - val_loss: 0.7416 - val_accuracy: 0.5380
Epoch 4/30
63/63 [==============================] - 4s 59ms/step - loss: 0.6209 - accuracy: 0.6665 - val_loss: 0.6357 - val_accuracy: 0.6280
Epoch 5/30
63/63 [==============================] - 6s 95ms/step - loss: 0.6213 - accuracy: 0.6760 - val_loss: 0.6113 - val_accuracy: 0.6780
Epoch 6/30
63/63 [==============================] - 4s 59ms/step - loss: 0.5890 - accuracy: 0.6845 - val_loss: 0.5981 - val_accuracy: 0.6700
Epoch 7/30
63/63 [==============================] - 7s 100ms/step - loss: 0.5519 - accuracy: 0.7290 - val_loss: 0.6622 - val_accuracy: 0.6110
Epoch 8/30
6

In [32]:
# to test the model
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 3s 48ms/step - loss: 0.6326 - accuracy: 0.6860
Test accuracy: 0.686


In [ ]:
Question 3:

In [43]:
# Creating directories and assiging images to training, validation and test directories
# Increasing the training sample from 2000 to 3500
import os
import shutil
import pathlib
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_OptimalTrainSamples1")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir,exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

# Training has 1500 samples, test has 1000 samples, and validation has 500 samples

make_subset("train", start_index=0, end_index=3500)
make_subset("validation", start_index=3500, end_index=4000)
make_subset("test", start_index=4000, end_index=5000)

In [44]:
# Building the model
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [45]:
#  the model
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])


In [46]:
# to use the callbacks function to monitor validation loss and running the model
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/30
200/200 [==============================] - 11s 47ms/step - loss: 0.7258 - accuracy: 0.5045 - val_loss: 0.6877 - val_accuracy: 0.6010
Epoch 2/30
200/200 [==============================] - 7s 33ms/step - loss: 0.6801 - accuracy: 0.5842 - val_loss: 0.6283 - val_accuracy: 0.6470
Epoch 3/30
200/200 [==============================] - 10s 48ms/step - loss: 0.6291 - accuracy: 0.6465 - val_loss: 0.5808 - val_accuracy: 0.6720
Epoch 4/30
200/200 [==============================] - 7s 33ms/step - loss: 0.5611 - accuracy: 0.7150 - val_loss: 0.5734 - val_accuracy: 0.7120
Epoch 5/30
200/200 [==============================] - 9s 46ms/step - loss: 0.5321 - accuracy: 0.7377 - val_loss: 0.5195 - val_accuracy: 0.7420
Epoch 6/30
200/200 [==============================] - 7s 36ms/step - loss: 0.4913 - accuracy: 0.7660 - val_loss: 0.5251 - val_accuracy: 0.7520
Epoch 7/30
200/200 [==============================] - 7s 32ms/step - loss: 0.4406 - accuracy: 0.7990 - val_loss: 0.5262 - val_accuracy: 0.76

In [47]:
# to test the model
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 4s 51ms/step - loss: 0.2605 - accuracy: 0.9020
Test accuracy: 0.902


In [48]:
# Creating directories and assiging images to training, validation and test directories
# Increasing the training sample from 3500 to 4500
import os
import shutil
import pathlib
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_OptimalTrainSamples1")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir,exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

# Training has 1500 samples, test has 1000 samples, and validation has 500 samples

make_subset("train", start_index=0, end_index=4500)
make_subset("validation", start_index=4500, end_index=5000)
make_subset("test", start_index=5000, end_index=6000)

In [49]:
# Building the model
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [50]:
#  the model
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [51]:
# to use the callbacks function to monitor validation loss and running the model
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/30
200/200 [==============================] - 10s 33ms/step - loss: 0.6970 - accuracy: 0.5337 - val_loss: 0.6703 - val_accuracy: 0.6260
Epoch 2/30
200/200 [==============================] - 9s 47ms/step - loss: 0.6638 - accuracy: 0.6112 - val_loss: 0.6123 - val_accuracy: 0.6650
Epoch 3/30
200/200 [==============================] - 7s 33ms/step - loss: 0.6220 - accuracy: 0.6518 - val_loss: 0.5531 - val_accuracy: 0.7090
Epoch 4/30
200/200 [==============================] - 11s 52ms/step - loss: 0.5852 - accuracy: 0.6950 - val_loss: 0.5330 - val_accuracy: 0.7150
Epoch 5/30
200/200 [==============================] - 7s 33ms/step - loss: 0.5283 - accuracy: 0.7430 - val_loss: 0.5106 - val_accuracy: 0.7500
Epoch 6/30
200/200 [==============================] - 8s 38ms/step - loss: 0.4904 - accuracy: 0.7700 - val_loss: 0.4913 - val_accuracy: 0.7670
Epoch 7/30
200/200 [==============================] - 8s 40ms/step - loss: 0.4427 - accuracy: 0.7893 - val_loss: 0.5060 - val_accuracy: 0.76

In [52]:
# to test the model
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 3s 35ms/step - loss: 0.4287 - accuracy: 0.8015
Test accuracy: 0.802


In [ ]:
PreTrained Network

In [ ]:
Question 4

In [84]:
# Creating directories and assiging images to training, validation and test directories
import os
import shutil
import pathlib

shutil.rmtree("cats_vs_dogs_small_Pretrained")

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_Pretrained")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

# Training has 1500 samples, test has 500 samples, and validation has 500 samples
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [85]:
# Loading pre-trained weights to VGG16 model
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [86]:
# Defining function to extract features and labels
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)
# Extracting the features and labels from datasets
train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)
train_features.shape

1/1 [==============================] - 0s 24ms/step


(10000, 5, 5, 512)

In [57]:
# Building the model
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

# Running the callback function to monitor validation loss
callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction.keras",
      save_best_only=True,
      monitor="val_loss")
]

# Training the model
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

Epoch 1/20
125/125 [==============================] - 2s 7ms/step - loss: 14.4286 - accuracy: 0.9402 - val_loss: 4.7613 - val_accuracy: 0.9710
Epoch 2/20
125/125 [==============================] - 1s 5ms/step - loss: 2.9448 - accuracy: 0.9775 - val_loss: 7.6235 - val_accuracy: 0.9620
Epoch 3/20
125/125 [==============================] - 1s 5ms/step - loss: 2.4627 - accuracy: 0.9835 - val_loss: 2.5617 - val_accuracy: 0.9830
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 0.9300 - accuracy: 0.9920 - val_loss: 3.2113 - val_accuracy: 0.9810
Epoch 5/20
125/125 [==============================] - 1s 6ms/step - loss: 0.9238 - accuracy: 0.9912 - val_loss: 3.8892 - val_accuracy: 0.9760
Epoch 6/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3178 - accuracy: 0.9930 - val_loss: 3.5977 - val_accuracy: 0.9760
Epoch 7/20
125/125 [==============================] - 1s 5ms/step - loss: 0.6581 - accuracy: 0.9950 - val_loss: 3.1940 - val_accuracy: 0.9790
Epoch

In [58]:
# Testing the model
test_model = keras.models.load_model("feature_extraction.keras")
test_loss, test_acc = test_model.evaluate(test_features, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 0s 3ms/step - loss: 0.3183 - accuracy: 0.9965
Test accuracy: 0.997


In [60]:
# Loading pre-trained weights to the VGG16 model
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
# Freezing the layers of the pretrained CNN
conv_base.trainable = False

# UnFreezing the layers of the pretrained CNN
conv_base.trainable = True
print("number of trainable weights "
      "before freezing the conv base:", len(conv_base.trainable_weights))
conv_base.trainable = False
print("number of trainable weights "
      "after freezing the conv base:", len(conv_base.trainable_weights))

number of trainable weights before freezing the conv base: 26
number of trainable weights after freezing the conv base: 0


In [61]:
 # Declaring Data Augumentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)
# Building the model and configuring it
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [62]:

# Using the callbacks function to monitor validation loss and running the model
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=60,
    validation_data=validation_dataset,
    callbacks=callbacks)



Epoch 1/60
200/200 [==============================] - 11s 46ms/step - loss: 14.5408 - accuracy: 0.9145 - val_loss: 4.6600 - val_accuracy: 0.9770
Epoch 2/60
200/200 [==============================] - 10s 46ms/step - loss: 7.7759 - accuracy: 0.9467 - val_loss: 3.4007 - val_accuracy: 0.9780
Epoch 3/60
200/200 [==============================] - 10s 51ms/step - loss: 5.1162 - accuracy: 0.9530 - val_loss: 2.5294 - val_accuracy: 0.9830
Epoch 4/60
200/200 [==============================] - 8s 37ms/step - loss: 4.1401 - accuracy: 0.9617 - val_loss: 3.4129 - val_accuracy: 0.9740
Epoch 5/60
200/200 [==============================] - 11s 54ms/step - loss: 3.8122 - accuracy: 0.9625 - val_loss: 4.7153 - val_accuracy: 0.9630
Epoch 6/60
200/200 [==============================] - 8s 37ms/step - loss: 2.5942 - accuracy: 0.9645 - val_loss: 3.0211 - val_accuracy: 0.9680
Epoch 7/60
200/200 [==============================] - 9s 45ms/step - loss: 2.2317 - accuracy: 0.9670 - val_loss: 1.4012 - val_accuracy: 0

In [63]:
# Testing the model
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")



63/63 [==============================] - 3s 40ms/step - loss: 0.1532 - accuracy: 0.9945
Test accuracy: 0.994


In [64]:
# Creating directories and assiging images to training, validation and test directories
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_PretrainedIncreasedSample")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir,exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

In [65]:
# Loading pre-trained weights to VGG16 model
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))
# conv_base.trainable = False

In [66]:
# Using the callbacks function to monitor validation loss and running the model
make_subset("train", start_index=0, end_index=1500)
make_subset("validation", start_index=1500, end_index=2000)
make_subset("test", start_index=2000, end_index=3000)

# Building the model
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

# Running the callback function to monitor validation loss
callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction.keras",
      save_best_only=True,
      monitor="val_loss")
]

# Training the model
history = model.fit(
    train_features, train_labels,
    epochs=30,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

Epoch 1/30
125/125 [==============================] - 2s 8ms/step - loss: 9.5280 - accuracy: 0.9410 - val_loss: 3.8333 - val_accuracy: 0.9720
Epoch 2/30
125/125 [==============================] - 1s 5ms/step - loss: 3.0908 - accuracy: 0.9797 - val_loss: 3.5169 - val_accuracy: 0.9810
Epoch 3/30
125/125 [==============================] - 0s 4ms/step - loss: 2.6990 - accuracy: 0.9830 - val_loss: 4.5592 - val_accuracy: 0.9780
Epoch 4/30
125/125 [==============================] - 0s 4ms/step - loss: 1.6183 - accuracy: 0.9872 - val_loss: 3.6109 - val_accuracy: 0.9770
Epoch 5/30
125/125 [==============================] - 0s 4ms/step - loss: 1.0756 - accuracy: 0.9923 - val_loss: 4.0270 - val_accuracy: 0.9750
Epoch 6/30
125/125 [==============================] - 0s 4ms/step - loss: 0.9620 - accuracy: 0.9942 - val_loss: 3.9606 - val_accuracy: 0.9830
Epoch 7/30
125/125 [==============================] - 1s 4ms/step - loss: 0.3828 - accuracy: 0.9958 - val_loss: 5.3838 - val_accuracy: 0.9760
Epoch 

In [67]:
# Testing the model
test_model = keras.models.load_model("feature_extraction.keras")
test_loss, test_acc = test_model.evaluate(test_features, test_labels)
print(f"Test accuracy: {test_acc:.3f}")


63/63 [==============================] - 0s 4ms/step - loss: 0.3032 - accuracy: 0.9945
Test accuracy: 0.994


In [91]:
# Creating directories and assiging images to training, validation and test directories
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_PretrainedoptimalSample")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir,exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

In [92]:
# Loading pre-trained weights to VGG16 model
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False




In [93]:
# Building the model
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [96]:
from tensorflow.keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

# Train the model with varying training sample sizes
sample_sizes = [3000,3500]
history_dict = []
for size in sample_sizes:
    # Set up the training subset
    make_subset("temp_train", start_index=0, end_index=size)
    make_subset("validation", start_index=size, end_index=size+500)
    make_subset("test", start_index=size+500, end_index=size+1500)
    train_dataset = image_dataset_from_directory(
      new_base_dir / "temp_train",
      image_size=(180, 180),
      batch_size=20)
    # Running the callback function to monitor validation loss
    callbacks = [
      keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction.keras",
      save_best_only=True,
      monitor="val_loss")]

    # Training the model
    history = model.fit(
      train_features, train_labels,
      epochs=20,
      validation_data=(val_features, val_labels),
      callbacks=callbacks)

   # Testing the model
    test_model = keras.models.load_model("feature_extraction.keras")
    test_loss, test_acc = test_model.evaluate(test_features, test_labels)
    print(f"Test accuracy: {test_acc:.3f}")

Found 14000 files belonging to 2 classes.
Epoch 1/20
313/313 [==============================] - 3s 9ms/step - loss: 0.0377 - accuracy: 0.9999 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
313/313 [==============================] - 2s 5ms/step - loss: 3.0007e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
313/313 [==============================] - 1s 4ms/step - loss: 0.0082 - accuracy: 0.9999 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
313/313 [==============================] - 1s 4ms/step - loss: 1.8523e-16 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
313/313 [==============================] - 1s 4ms/step - loss: 0.0044 - accuracy: 0.9998 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
313/313 [==============================] - 1s 4ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/20
313/313 [==============================] - 1s 4ms/step - loss: 